In [61]:
%reset -f
%load_ext autoreload
%autoreload 2

import Odin as backend

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
# validation test for incompressible flow

geom = backend.createGeometry( 'rectangle', [5,2] )
mesh = backend.createMesh( geom, res=1 )

# make just a list, give a name as optional parameter to flow model
myFlowModels = [
    backend.FlowModels.IncompressibleMomentumComp(depVariableName='u', orientation=0, otherVelocityFieldName='v', pressureFieldName='p', kinViscosityName='nu')
]

sim1 = backend.Simulation(myFlowModels, mesh, geom )

myFields = sim1.getFieldRegistry()

print(myFields)

U = myFields['u']
#U.setInternalValue([0,0])
U.u = 0.0
U.v = 0.0

#U.setBoundaryCondition('top', [1,0])
U.setBoundaryCondition('top', 0)
U.setBoundaryCondition('bottom', 0)
U.setBoundaryCondition('left', 1)
U.setBoundaryCondition('right', 0)
myFields['rho'] = 1000.0

sim1.solve('u')

sim1.display(U)

{'u': <ScalarField.scalarField object at 0x7f8d53da0310>, 'v': <ScalarField.scalarField object at 0x7f8d53da09a0>, 'p': <ScalarField.scalarField object at 0x7f8d53da0610>, 'nu': 0.0}


ValueError: could not broadcast input array from shape (2,5) into shape (2,6)

In [ ]:
# shutting the interpreter down, so I can a fresh instance next time.
# my modules are automatically checked for updates
import os
os._exit(00)